# Import Statements and Magic Commands

In [1]:
import datetime
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn import tree
#from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import GradientBoostingRegressor
#from xgboost import XGBRegressor
#from sklearn import linear_model
#from sklearn.metrics import mean_squared_log_error
# WHEN CHECKING THE MODEL: use as np.sqrt(mean_squared_log_error( y_test, predictions ))
from pandas.tseries.holiday import USFederalHolidayCalendar
%matplotlib inline

---
# Load the Data

### Building and Weather Data

In [6]:
# Note: These assume that the data has been saved to a subdirectory named "energy"
bldgData = pd.read_csv("energy/building_metadata.csv")
weatherTrain = pd.read_csv("energy/weather_train.csv")
weatherTest = pd.read_csv("energy/weather_test.csv")
allWeather = weatherTrain.append(weatherTest)

### Training Data

In [7]:
energyTrain = pd.read_csv("energy/train.csv")

### Test Data
Warning: This will use a lot of memory! My virtual machine is sitting at 5.5 GB of RAM utilization right now.

In [8]:
energyTest = pd.read_csv("energy/test.csv")

### Explore the Data

In [4]:
#print(bldgData.head(5))
#print(allWeather.head(5))
#print(allWeather.tail(5))
print(energyTrain.head(5))

   building_id  meter            timestamp  meter_reading
0            0      0  2016-01-01 00:00:00            0.0
1            1      0  2016-01-01 00:00:00            0.0
2            2      0  2016-01-01 00:00:00            0.0
3            3      0  2016-01-01 00:00:00            0.0
4            4      0  2016-01-01 00:00:00            0.0


---
# 2.a. Decision Tree Regressor - Version 1

In [8]:
trainJoinBldg = pd.merge(energyTrain, bldgData, on='building_id')
fullTraining = pd.merge(trainJoinBldg, allWeather, on=['site_id', 'timestamp'])

del energyTrain
del trainJoinBldg
del bldgData
del weatherTrain
del weatherTest
del allWeather

In [9]:
print(fullTraining.shape)
fullTraining = fullTraining[fullTraining.meter_reading != 0.0]
print(fullTraining.shape)

slimTraining = fullTraining[['building_id', 'meter', 'timestamp', 'meter_reading']]

del fullTraining

print(slimTraining.shape)

(20125605, 16)
(18257718, 16)
(18257718, 5)


In [10]:
slimTraining["hour"] = ""
slimTraining["month"] = ""
slimTraining["dayOfWeek"] = ""

for i, row in slimTraining.iterrows():
    slimTraining.at[i,'hour'] = row['timestamp'].split(" ")[1]
    date = datetime.datetime.strptime(row['timestamp'], '%Y-%m-%d %H:%M:%S')
    slimTraining.at[i,'month'] = date.strftime('%b')
    slimTraining.at[i,'dayOfWeek'] = str(date.weekday())
    
slimTraining.drop("timestamp", axis=1, inplace=True)

In [13]:
print(slimTraining.shape)
slimTraining.to_csv("custom/hourMonthDayofweek.csv")

(18257718, 7)


In [2]:
slimTraining = pd.read_csv("custom/hourMonthDayofweek.csv")
slimTraining.head(5)

,Unnamed: 0,building_id,meter,meter_reading,air_temperature,hour,month,dayOfWeek
0,45,46,0,53.2397,25.0,00:00:00,Jan,4
1,72,74,0,43.0013,25.0,00:00:00,Jan,4
2,91,93,0,52.4206,25.0,00:00:00,Jan,4
3,123,20,0,91.9886,24.4,01:00:00,Jan,4
4,148,46,0,53.6492,24.4,01:00:00,Jan,4


In [3]:
buildingMeters = []

for row in slimTraining.itertuples(index=False):
    if (row.building_id, row.meter) not in buildingMeters:
        buildingMeters.append((row.building_id, row.meter))
    
print(len(buildingMeters))

2380


In [4]:
readyToTrain = pd.get_dummies(slimTraining, columns=['hour', 'month', 'dayOfWeek'])

In [5]:
readyToTrain.head(5)

,Unnamed: 0,building_id,meter,meter_reading,air_temperature,hour_00:00:00,hour_01:00:00,hour_02:00:00,hour_03:00:00,hour_04:00:00,...,month_Nov,month_Oct,month_Sep,dayOfWeek_0,dayOfWeek_1,dayOfWeek_2,dayOfWeek_3,dayOfWeek_4,dayOfWeek_5,dayOfWeek_6
0,45,46,0,53.2397,25.0,1,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
1,72,74,0,43.0013,25.0,1,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
2,91,93,0,52.4206,25.0,1,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
3,123,20,0,91.9886,24.4,0,1,0,0,0,...,0,0,0,0,0,0,0,1,0,0
4,148,46,0,53.6492,24.4,0,1,0,0,0,...,0,0,0,0,0,0,0,1,0,0


In [6]:
trainedModels = {}

counter = 0
oldPercent = 0.
for building_id, meter in buildingMeters:
    buildingMeterData = readyToTrain[(readyToTrain['building_id'] == building_id) & (readyToTrain['meter'] == meter)]
    # TRAIN A MODEL USING buildingMeterData
    y = buildingMeterData['meter_reading']
    X = buildingMeterData[[
        'hour_00:00:00',
        'hour_01:00:00',
        'hour_02:00:00',
        'hour_03:00:00',
        'hour_04:00:00',
        'hour_05:00:00',
        'hour_06:00:00',
        'hour_07:00:00',
        'hour_08:00:00',
        'hour_09:00:00',
        'hour_10:00:00',
        'hour_11:00:00',
        'hour_12:00:00',
        'hour_13:00:00',
        'hour_14:00:00',
        'hour_15:00:00',
        'hour_16:00:00',
        'hour_17:00:00',
        'hour_18:00:00',
        'hour_19:00:00',
        'hour_20:00:00',
        'hour_21:00:00',
        'hour_22:00:00',
        'hour_23:00:00',
        'month_Jan',
        'month_Feb',
        'month_Mar',
        'month_Apr',
        'month_May',
        'month_Jun',
        'month_Jul',
        'month_Aug',
        'month_Sep',
        'month_Oct',
        'month_Nov',
        'month_Dec',
        'dayOfWeek_0',
        'dayOfWeek_1',
        'dayOfWeek_2',
        'dayOfWeek_3',
        'dayOfWeek_4',
        'dayOfWeek_5',
        'dayOfWeek_6'
    ]]
    dtRegressor = tree.DecisionTreeRegressor()
    trainedModels[(building_id, meter)] = dtRegressor.fit(X, y)
    counter += 1
    percent = 100. * counter / 2380
    if percent - oldPercent > 5:
        print('{0:.2f}'.format(percent))
        oldPercent = percent

print(len(trainedModels))

5.04
10.08
15.13
20.17
25.21
30.25
35.29
40.34
45.38
50.42
55.46
60.50
65.55
70.59
75.63
80.67
85.71
90.76
95.80
2380


In [7]:
#print(trainedModels[(93,0)].feature_importances_)
del slimTraining
del readyToTrain

In [9]:
print(energyTest.shape)

(41697600, 4)


In [14]:
# Note: Create a subdirectory named "results" before running
dtrResults = "row_id,meter_reading\n"

counter = 0
oldPercent = 0.
for row in energyTest.itertuples(index=False):
    row_id = str(row.row_id)
    building_id = row.building_id
    meter = row.meter
    dateStr, hour = row.timestamp.split(" ")
    
    date = datetime.datetime.strptime(dateStr, '%Y-%m-%d')
    
    month = date.strftime('%b')
    dayOfWeek = str(date.weekday())
    
    features = pd.Series(0, index=[
        'hour_00:00:00',
        'hour_01:00:00',
        'hour_02:00:00',
        'hour_03:00:00',
        'hour_04:00:00',
        'hour_05:00:00',
        'hour_06:00:00',
        'hour_07:00:00',
        'hour_08:00:00',
        'hour_09:00:00',
        'hour_10:00:00',
        'hour_11:00:00',
        'hour_12:00:00',
        'hour_13:00:00',
        'hour_14:00:00',
        'hour_15:00:00',
        'hour_16:00:00',
        'hour_17:00:00',
        'hour_18:00:00',
        'hour_19:00:00',
        'hour_20:00:00',
        'hour_21:00:00',
        'hour_22:00:00',
        'hour_23:00:00',
        'month_Jan',
        'month_Feb',
        'month_Mar',
        'month_Apr',
        'month_May',
        'month_Jun',
        'month_Jul',
        'month_Aug',
        'month_Sep',
        'month_Oct',
        'month_Nov',
        'month_Dec',
        'dayOfWeek_0',
        'dayOfWeek_1',
        'dayOfWeek_2',
        'dayOfWeek_3',
        'dayOfWeek_4',
        'dayOfWeek_5',
        'dayOfWeek_6'
    ])
    
    features['hour_' + hour] = 1
    features['month_' + month] = 1
    features['dayOfWeek_' + dayOfWeek] = 1
    
    pred = trainedModels[(building_id, meter)].predict([features])[0]

    dtrResults += row_id + "," + '{0:.4f}'.format(pred) + "\n"
    
    counter += 1
    percent = 100. * counter / 41697600
    if percent - oldPercent > 5:
        print('{0:.2f}'.format(percent))
        oldPercent = percent
    
outFile = open("results/decisionTreeRegressor.csv","w")
outFile.write(dtrResults)
outFile.close()

5.00
10.00
15.00
20.00
25.00
30.00
35.00
40.00
45.00
50.00
55.00
60.00
65.00
70.00
75.00
80.00
85.00
90.00
95.00


### Kaggle RMSLE score for the Decision Tree Regressor, Version 1 = 1.38

---
# 2.b. Decision Tree Regressor - Version 2

In [2]:
slimTraining = pd.read_csv("custom/hourMonthDayofweek.csv")
slimTraining.head(5)

,Unnamed: 0,building_id,meter,meter_reading,air_temperature,hour,month,dayOfWeek
0,45,46,0,53.2397,25.0,00:00:00,Jan,4
1,72,74,0,43.0013,25.0,00:00:00,Jan,4
2,91,93,0,52.4206,25.0,00:00:00,Jan,4
3,123,20,0,91.9886,24.4,01:00:00,Jan,4
4,148,46,0,53.6492,24.4,01:00:00,Jan,4


In [3]:
buildingMeters = []

for row in slimTraining.itertuples(index=False):
    if (row.building_id, row.meter) not in buildingMeters:
        buildingMeters.append((row.building_id, row.meter))
    
print(len(buildingMeters))

2380


In [4]:
readyToTrain = pd.get_dummies(slimTraining, columns=['hour', 'month', 'dayOfWeek'])

In [5]:
readyToTrain.head(5)

,Unnamed: 0,building_id,meter,meter_reading,air_temperature,hour_00:00:00,hour_01:00:00,hour_02:00:00,hour_03:00:00,hour_04:00:00,...,month_Nov,month_Oct,month_Sep,dayOfWeek_0,dayOfWeek_1,dayOfWeek_2,dayOfWeek_3,dayOfWeek_4,dayOfWeek_5,dayOfWeek_6
0,45,46,0,53.2397,25.0,1,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
1,72,74,0,43.0013,25.0,1,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
2,91,93,0,52.4206,25.0,1,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
3,123,20,0,91.9886,24.4,0,1,0,0,0,...,0,0,0,0,0,0,0,1,0,0
4,148,46,0,53.6492,24.4,0,1,0,0,0,...,0,0,0,0,0,0,0,1,0,0


In [8]:
print(readyToTrain.shape)
readyToTrain = readyToTrain[pd.notnull(readyToTrain['air_temperature'])]
print(readyToTrain.shape)

(18257718, 48)
(18252676, 48)


In [9]:
trainedModels = {}

counter = 0
oldPercent = 0.
for building_id, meter in buildingMeters:
    buildingMeterData = readyToTrain[(readyToTrain['building_id'] == building_id) & (readyToTrain['meter'] == meter)]
    # TRAIN A MODEL USING buildingMeterData
    y = buildingMeterData['meter_reading']
    X = buildingMeterData[[
        'air_temperature',
        'hour_00:00:00',
        'hour_01:00:00',
        'hour_02:00:00',
        'hour_03:00:00',
        'hour_04:00:00',
        'hour_05:00:00',
        'hour_06:00:00',
        'hour_07:00:00',
        'hour_08:00:00',
        'hour_09:00:00',
        'hour_10:00:00',
        'hour_11:00:00',
        'hour_12:00:00',
        'hour_13:00:00',
        'hour_14:00:00',
        'hour_15:00:00',
        'hour_16:00:00',
        'hour_17:00:00',
        'hour_18:00:00',
        'hour_19:00:00',
        'hour_20:00:00',
        'hour_21:00:00',
        'hour_22:00:00',
        'hour_23:00:00',
        'month_Jan',
        'month_Feb',
        'month_Mar',
        'month_Apr',
        'month_May',
        'month_Jun',
        'month_Jul',
        'month_Aug',
        'month_Sep',
        'month_Oct',
        'month_Nov',
        'month_Dec',
        'dayOfWeek_0',
        'dayOfWeek_1',
        'dayOfWeek_2',
        'dayOfWeek_3',
        'dayOfWeek_4',
        'dayOfWeek_5',
        'dayOfWeek_6'
    ]]
    dtRegressor = tree.DecisionTreeRegressor()
    trainedModels[(building_id, meter)] = dtRegressor.fit(X, y)
    counter += 1
    percent = 100. * counter / 2380
    if percent - oldPercent > 5:
        print('{0:.2f}'.format(percent))
        oldPercent = percent

print(len(trainedModels))

5.04
10.08
15.13
20.17
25.21
30.25
35.29
40.34
45.38
50.42
55.46
60.50
65.55
70.59
75.63
80.67
85.71
90.76
95.80
2380


In [12]:
print(trainedModels[(46,0)].feature_importances_)
#del slimTraining
#del readyToTrain

[3.93925253e-01 1.03464868e-02 7.34604696e-03 4.23519032e-03
 3.11844798e-03 3.81030062e-03 3.08926658e-03 5.25449611e-03
 9.26175569e-03 7.13031299e-03 7.71541332e-03 7.72026709e-03
 7.75983110e-03 9.52487387e-03 8.98468725e-03 9.25170509e-03
 1.14937883e-02 9.79922901e-03 8.38481383e-03 6.63852892e-03
 4.09198792e-02 7.50164560e-03 6.10773359e-03 7.39284342e-03
 9.56555767e-03 1.76016391e-01 7.97085490e-03 2.87905758e-04
 0.00000000e+00 9.70105676e-03 3.25079957e-03 2.44662761e-03
 3.87167277e-02 2.78061799e-03 5.28982929e-03 4.08859964e-03
 4.82367443e-03 1.26597327e-02 1.80941843e-02 2.56455750e-02
 2.20017978e-02 1.93342865e-02 2.17269423e-02 1.88860431e-02]


In [13]:
energyTest = pd.read_csv("energy/test.csv")
print(energyTest.shape)

bldgData = pd.read_csv("energy/building_metadata.csv")
weatherTest = pd.read_csv("energy/weather_test.csv")

testJoinBldg = pd.merge(energyTest, bldgData, on='building_id')
fullTest = pd.merge(testJoinBldg, weatherTest, on=['site_id', 'timestamp'])

del energyTest
del testJoinBldg
del bldgData
del weatherTest

print(fullTest.shape)

(41697600, 4)
(41498571, 16)


In [15]:
print(fullTest.shape)
fullTest = fullTest[pd.notnull(fullTest['air_temperature'])]
print(fullTest.shape)

(41498571, 16)
(41475699, 16)


In [16]:
# Note: Create a subdirectory named "results" before running
dtrResults = "row_id,meter_reading\n"

counter = 0
oldPercent = 0.
for row in fullTest.itertuples(index=False):
    row_id = str(row.row_id)
    building_id = row.building_id
    meter = row.meter
    air_temperature = row.air_temperature
    dateStr, hour = row.timestamp.split(" ")
    
    date = datetime.datetime.strptime(dateStr, '%Y-%m-%d')
    
    month = date.strftime('%b')
    dayOfWeek = str(date.weekday())
    
    features = pd.Series(0, index=[
        'air_temperature',
        'hour_00:00:00',
        'hour_01:00:00',
        'hour_02:00:00',
        'hour_03:00:00',
        'hour_04:00:00',
        'hour_05:00:00',
        'hour_06:00:00',
        'hour_07:00:00',
        'hour_08:00:00',
        'hour_09:00:00',
        'hour_10:00:00',
        'hour_11:00:00',
        'hour_12:00:00',
        'hour_13:00:00',
        'hour_14:00:00',
        'hour_15:00:00',
        'hour_16:00:00',
        'hour_17:00:00',
        'hour_18:00:00',
        'hour_19:00:00',
        'hour_20:00:00',
        'hour_21:00:00',
        'hour_22:00:00',
        'hour_23:00:00',
        'month_Jan',
        'month_Feb',
        'month_Mar',
        'month_Apr',
        'month_May',
        'month_Jun',
        'month_Jul',
        'month_Aug',
        'month_Sep',
        'month_Oct',
        'month_Nov',
        'month_Dec',
        'dayOfWeek_0',
        'dayOfWeek_1',
        'dayOfWeek_2',
        'dayOfWeek_3',
        'dayOfWeek_4',
        'dayOfWeek_5',
        'dayOfWeek_6'
    ])
    
    features['air_temperature'] = air_temperature
    features['hour_' + hour] = 1
    features['month_' + month] = 1
    features['dayOfWeek_' + dayOfWeek] = 1
    
    pred = trainedModels[(building_id, meter)].predict([features])[0]

    dtrResults += row_id + "," + '{0:.4f}'.format(pred) + "\n"
    
    counter += 1
    percent = 100. * counter / 41475699
    if percent - oldPercent > 1:
        print('{0:.2f}'.format(percent))
        oldPercent = percent
    
outFile = open("results/decisionTreeRegressor02.csv","w")
outFile.write(dtrResults)
outFile.close()

1.00
2.00
3.00
4.00
5.00
6.00
7.00
8.00
9.00
10.00
11.00
12.00
13.00
14.00
15.00
16.00
17.00
18.00
19.00
20.00
21.00
22.00
23.00
24.00
25.00
26.00
27.00
28.00
29.00
30.00
31.00
32.00
33.00
34.00
35.00
36.00
37.00
38.00
39.00
40.00
41.00
42.00
43.00
44.00
45.00
46.00
47.00
48.00
49.00
50.00
51.00
52.00
53.00
54.00
55.00
56.00
57.00
58.00
59.00
60.00
61.00
62.00
63.00
64.00
65.00
66.00
67.00
68.00
69.00
70.00
71.00
72.00
73.00
74.00
75.00
76.00
77.00
78.00
79.00
80.00
81.00
82.00
83.00
84.00
85.00
86.00
87.00
88.00
89.00
90.00
91.00
92.00
93.00
94.00
95.00
96.00
97.00
98.00
99.00


In [17]:
# Merge DTR2 results with DTR1 (to fill in missing predictions)
dtr1results = pd.read_csv("results/decisionTreeRegressor.csv")
dtr2results = pd.read_csv("results/decisionTreeRegressor02.csv")

dtr2results.rename(columns={"meter_reading":"meter_reading2"}, inplace=True)
dtr2results.head(5)


,row_id,meter_reading2
0,0,37.5408
1,1,14.1973
2,2,1.6381
3,3,336.5020
4,4,86.0026


In [18]:
mergedResults = pd.merge(dtr1results, dtr2results, on='row_id', how='left')
mergedResults.tail(5)

,row_id,meter_reading,meter_reading2
41697595,41697595,4.4750,NaN
41697596,41697596,5.3375,NaN
41697597,41697597,10.7750,NaN
41697598,41697598,170.0938,NaN
41697599,41697599,4.4625,NaN


In [19]:
mergedResults.head(10)

,row_id,meter_reading,meter_reading2
0,0,37.5408,37.5408
1,1,13.1052,14.1973
2,2,1.6381,1.6381
3,3,385.8170,336.5020
4,4,86.0026,86.0026
5,5,32.8311,4.4366
6,6,91.5723,99.1077
7,7,489.9072,526.9370
8,8,981.3908,886.4180
9,9,379.5888,478.1330


In [20]:
mergedResults.drop("meter_reading", axis=1, inplace=True)
mergedResults.rename(columns={"meter_reading2":"meter_reading"}, inplace=True)
mergedResults.head(10)

,row_id,meter_reading
0,0,37.5408
1,1,14.1973
2,2,1.6381
3,3,336.5020
4,4,86.0026
5,5,4.4366
6,6,99.1077
7,7,526.9370
8,8,886.4180
9,9,478.1330


In [21]:
mergedResults.tail(5)

,row_id,meter_reading
41697595,41697595,NaN
41697596,41697596,NaN
41697597,41697597,NaN
41697598,41697598,NaN
41697599,41697599,NaN


In [22]:
dtr1results.update(mergedResults)
print(dtr1results.shape)

(41697600, 2)


In [23]:
dtr1results.head(10)

,row_id,meter_reading
0,0,37.5408
1,1,14.1973
2,2,1.6381
3,3,336.5020
4,4,86.0026
5,5,4.4366
6,6,99.1077
7,7,526.9370
8,8,886.4180
9,9,478.1330


In [24]:
dtr1results.tail(10)

,row_id,meter_reading
41697590,41697590,297.8750
41697591,41697591,298.0062
41697592,41697592,84.1000
41697593,41697593,35.2910
41697594,41697594,80.0625
41697595,41697595,4.4750
41697596,41697596,5.3375
41697597,41697597,10.7750
41697598,41697598,170.0938
41697599,41697599,4.4625


In [26]:
dtr1results.to_csv("results/decisionTreeRegressor02_merged.csv", index=False)

### Kaggle RMSLE score for the Decision Tree Regressor, Version 2 = 1.34